**Connect and authorize google drive with google colab:**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

# 1. Import Libraries



In [ ]:
import os
import numpy   as np
import pandas  as pd
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline  

# 2. Note on the analysis

**I will consider the data of flights ONLY in January and February 2017. Here I used Google Colab, and it may consume time, so I will use the data of the 02 months for the analysis.**

**This dataset is composed by the following variables:**

YEAR: 2017

QUARTER: 1-4

MONTH 1-12

DAY_OF_MONTH: 1-31

DAY_OF_WEEK: 1-7

OP_UNIQUE_CARRIER: Two letter carrier abbreviation

TAIL_NUM: Plane tail number

OP_CARRIER_FL_NUM: Flight number

ORIGIN: Airport codes for origin

DEST: Airport codes for destination

CRS_DEP_TIME: Scheduled departure time, local timezone

DEP_TIME: Departure times, local timezone

DEP_DELAY: Departure delay, in minutes, Negative times represent early departures

TAXI_OUT: taxi out time in minutes

TAXI_IN: taxi in time, in minutes

CRS_ARR_TIME: Scheduled arrival time

ARR_TIME Arrival times, local timezone

ARR_DELAY: Arrival delay, in minutes, Negative times represent early arrivals

CANCELLED: the flight was cancelled or not

CANCELLATION_CODE: reason for cancellation (A = carrier, B = weather, C = NAS, D = security)

DIVERTED: 1 = yes, 0 = no

CRS_ELAPSED_TIME: Scheduled elapsed time in minutes

ACTUAL_ELAPSED_TIME: Actual elapsed time in minutes

AIR_TIME: Amount of time spent in the air, in minutes

DISTANCE: Distance flown, in miles

CARRIER_DELAY: 	in minutes

WEATHER_DELAY: in minutes

NAS_DELAY: in minutes

SECURITY_DELAY: in minutes

LATE_AIRCRAFT_DELAY: in minutes



# 3. Data directory

In [ ]:
data_dir = '/content/gdrive/My Drive/data'
%cd '/content/gdrive/My Drive/data'

current_dir = os.getcwd()
print(current_dir)
data_path = os.path.join(data_dir, 'flights', '')
print(data_path)

# 4. Read data sets

## Read carrier data

In [ ]:
# Read carrier data from csv
carriers = pd.read_csv( os.path.join(data_path, 'carriers.csv') )
print('Total number of unique carriers: ', len(carriers))
carriers.head()

In [ ]:
carriers.rename({'Description': 'AIRLINE_NAME', 'Code' : 'code'}, axis=1, inplace=True)  # new method
carriers.head()

In [ ]:
# Checking the null values
carriers.isnull().sum()

In [ ]:
# Checking the null values
carriers[carriers.code.isnull()]

In [ ]:
# Google search and correct the code for North American Airlines
carriers.at[873, 'code'] = 'NA'

In [ ]:
# Checking the null values
carriers[carriers.code.isnull()]

In [ ]:
# Checking the null values
carriers[carriers.AIRLINE_NAME == 'North American Airlines']

## Read airport data

In [ ]:
# Read airport data from csv
airports = pd.read_csv( os.path.join(data_path, 'airports.csv') )
airports.head()

In [ ]:
# Checking the null values
airports.isnull().sum()

In [ ]:
# Checking the null values
airports[airports.city.isnull()]

In [ ]:
# Google search and correct the code for North American Airlines
airports.at[1136, 'city']  = 'Carlsbad'
airports.at[1136, 'state'] = 'CA'

airports.at[1715, 'city']  = 'Beaufort County'
airports.at[1715, 'state'] = 'SC'

airports.at[2251, 'city']  = 'Ward County'
airports.at[2251, 'state'] = 'ND'

airports.at[2312, 'city']  = 'Marquette County'
airports.at[2312, 'state'] = 'MI'

airports.at[2752, 'city']  = 'Box Elder'
airports.at[2752, 'state'] = 'SD'

airports.at[2759, 'city']  = 'Emerado'
airports.at[2759, 'state'] = 'ND'

airports.at[2794, 'city']  = 'Prachin Buri'
airports.at[2794, 'state'] = 'Prachin Buri'

airports.at[2795, 'city']  = 'Babelthoup'
airports.at[2795, 'state'] = 'Babelthoup'

airports.at[2900, 'city']  = 'Centre County'
airports.at[2900, 'state'] = 'PA'

airports.at[2964, 'city']  = 'Spokane'
airports.at[2964, 'state'] = 'WA'

airports.at[3001, 'city']  = 'Tinian'
airports.at[3001, 'state'] = 'Northern Mariana'

airports.at[3355, 'city']  = ' Yap main island'
airports.at[3355, 'state'] = 'Yap'


In [ ]:
# Checking the null values
airports[airports.city.isnull()]

In [ ]:
# Checking the null values
airports.isnull().sum()

## Read flight data

### Data for January 2017

In [ ]:
df1 = pd.read_csv(os.path.join(data_path, '2017_jan.csv') )
df1.head()

### Data for February 2017

In [ ]:
df2 = pd.read_csv(os.path.join(data_path, '2017_feb.csv') )
df2.head()

### Merge the data of the 02 months

In [ ]:
# Concatenating the two data=frames
frames = [df1, df2]
# df = pd.concat(frames, keys=['jan', 'feb'])
df = pd.concat(frames)

In [ ]:
# Print some infor
print('Shape: ', df.shape)
print('Number of columns/features: ', len(df.columns))
print('Columns/features: \n', df.columns)
print('Data types of features: \n', df.dtypes)

# 5. Analysis

## 5.1 Check null values, remove cancelled and diverted flights

I'll remove cancelled and diverted flights because they're unecessary for this analysis.

In [ ]:
# Check null values
df.isnull().sum()

In [ ]:
# Remove unnecessary columns
df.drop(['CANCELLATION_CODE', 'Unnamed: 30'], axis=1, inplace=True)

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
# Calculate the percentage of cancelled flights
df['CANCELLED'].value_counts(normalize=True) * 100

In [ ]:
plt.figure(figsize=(6,6))
colors = ['g', 'r']
labels = ['Not Cancelled', 'Cancelled']
explode = (0,0.8)
plt.pie(df['CANCELLED'].value_counts(normalize=True) * 100, colors=colors, labels=labels,
explode=explode, autopct='%1.1f%%', shadow=True)
plt.title('Percentage of cancelled flights')
plt.show()

Pie char for cancelled and departed flights, only 1.8\% of the flights are cancelled.

In [ ]:
# Remove cancelled flights, they're not necessary for this analysis
df = df[df.CANCELLED != 1]

# Check if the cancelled flights have been removed
df['CANCELLED'].value_counts(normalize=True) * 100

In [ ]:
# Calculate the percentage of diverted flights
df['DIVERTED'].value_counts(normalize=True) * 100

In [ ]:
plt.figure(figsize=(6,6))
colors = ['g', 'r']
labels = ['Not diverted', 'Diverted']
explode = (0,0.8)
plt.pie(df['DIVERTED'].value_counts(normalize=True) * 100, colors=colors, labels=labels,
explode=explode, autopct='%1.1f%%', shadow=False)
plt.title('Percentage of diverted flights')
plt.show()

Pie char for diverted and normal flights, only 0.3\% of the flights are diverted.

In [ ]:
# Also remove diverted flights
df = df[df.DIVERTED != 1]
# df.isnull().sum(axis=0)

# Check if the diverted flights have been removed
df['DIVERTED'].value_counts(normalize=True) * 100

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
# The LATE_AIRCRAFT_DELAY (in minutes) is the delay caused by the aircraft
# (i.e. an aircraft which is scheduled to depart at a specific timing arrived late from its previous flight and hence the delay spilled over to the next departure)
# A null value in the LATE_AIRCRAFT_DELAY would mean that there is no delay incurred from the late arrival of the previous flight.
# I'll set all null values to be 0.
df.fillna(value={'LATE_AIRCRAFT_DELAY' : 0}, inplace=True)

In [ ]:
# Similarly, set all null values in CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY to be 0
df.fillna(value={'CARRIER_DELAY' : 0, 'WEATHER_DELAY' : 0, 'NAS_DELAY' : 0, 'SECURITY_DELAY' : 0}, inplace=True)

In [ ]:
# Check null values
df.isnull().sum(axis=0)

In [ ]:
# Check: No DIVERTED flights anymore
df.DIVERTED.unique().tolist()

In [ ]:
# Check: No CANCELLED flights anymore
df.CANCELLED.unique().tolist()

In [ ]:
# Dropping DIVERTED & CANCELLED columns
df.drop(['DIVERTED', 'CANCELLED'], axis=1, inplace=True)
df.head()

In [ ]:
# OK! No more null/nan values
# Check the datatypes
df.info()

## 5.2 Add carrier descriptions

In [ ]:
# Add carrier descriptions
df = pd.merge(df, carriers, how='left', left_on='OP_UNIQUE_CARRIER', right_on='code')
df.head()

In [ ]:
df.drop(['code'], axis=1, inplace=True)

## 5.3 Delay Analysis

In [ ]:
# Set the delay threshold
delay_thresh = 30 # minutes

I computed the summary statistics of the departure and arrival delays of all flights to gain a broad understanding of the data.

*   Departure delay is the difference between the official departure time and the actual departure time of the flight measured in minutes.

*   Arrival delay is the difference between the official arrival time and the actual departure time of the flight measured in minutes.

*   Positive values for delay indicate that the flight was delayed, negative values indicate the flight departed/arrived early

In [ ]:
# 'ARR_DELAY', 'DEP_DELAY', 'AIR_TIME' in [minutes]
# DISTANCE in [km]
df[['ARR_DELAY', 'DEP_DELAY']].describe()


The average arrival delay is only around 4 minutes. In addition, the median value is -6 minutes, suggesting the majority of flights actually arrive earlier than their expected time of arrival.

To illustrate this, I created a pie chart depicting the percentage of flights for each interval of arrival delay.

In [ ]:
labels = ['Early > 30 min',' Early < 30 min',' Late < 30 min', 'Late < 2 hours', 'Late > 2 hours']
bins   = [-100, -30, 0, 30, 120, 1945]

df['ARR_DELAY_BINS'] = pd.cut(df['ARR_DELAY'], bins=bins, labels=labels)
a = df.groupby('ARR_DELAY_BINS').size()
print(a)
# a.plot.pie(figsize=(6,6))

plt.figure(figsize=(7,7))
colors = ['r', 'y', 'g', 'lightblue', 'orange']
explode = (0, 0, 0, 0, 0.5)
plt.pie(a, colors=colors, labels=labels,
explode=explode, autopct='%1.1f%%', shadow=False)
plt.title('Percentage of delayed arrival flights for each interval')
plt.show()

# Drop the newly added ARR_DELAY_BINS column
df.drop(['ARR_DELAY_BINS'], axis=1, inplace=True)

As shown, ~65% of flights arrive early and only ~12% of flights arrive more than 30 minutes late.

The vast majority of flights arrive ahead of schedule. Moreover, only around 12% of flights are late by more than 30 minutes.

In [ ]:
# ax = df.hist(column=['ARR_DELAY', 'DEP_DELAY'], bins=100, grid=False, figsize=(12,8), layout=(1,2), color='#86bf91', zorder=2, rwidth=0.9, density=1)
ax = df.hist(column=['ARR_DELAY', 'DEP_DELAY'], bins=150, grid=False, figsize=(12,8), layout=(1,2), color='#86bf91', zorder=2, rwidth=0.99)

ax = ax[0]
for i,x in enumerate(ax):

    # Despine
    x.spines['right'].set_visible(False)
    x.spines['top'].set_visible(False)
    x.spines['left'].set_visible(False)

    # Switch off ticks
    x.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

    # Draw horizontal axis lines
    vals = x.get_yticks()
    for tick in vals:
        x.axhline(y=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

    # Remove title
    x.set_title("")

    # Set x-axis label
    xlabel = "ARRIVAL DELAY (minutes)" if i == 1 else "DEPARTURE DELAY (minutes)"
    x.set_xlabel(xlabel, labelpad=20, weight='bold', size=12)

    # Set y-axis label
    x.set_ylabel("Counts", labelpad=20, weight='bold', size=12)

    # Set log-scale for y-axis
    # x.set_yscale('log')

    x.set_xlim(-75, 300)

Histograms of departure and arrival delays (number of flights vs minutes). The average arrival delay is only around 4 minutes, the average departure delay is only around 9 minutes.

## 5.4 Causes of arrival delay

### a. Carrier delay

In [ ]:
carrier_delay =  df[df['CARRIER_DELAY'] > 0]
carrier_delay = carrier_delay.reset_index()

carrier_delay.CARRIER_DELAY.unique().tolist()[:15]

carrier_delay = carrier_delay.drop('index',axis=1)
carrier_delay.head()

### b. Weather delay

In [ ]:
weather_delay =  df[df['WEATHER_DELAY'] > 0]
weather_delay = weather_delay.reset_index()

weather_delay = weather_delay.drop('index', axis=1)
weather_delay.head()

### c. Security delay

In [ ]:
security_delay =  df[df['SECURITY_DELAY'] > 0]
security_delay = security_delay.reset_index()

security_delay = security_delay.drop('index', axis=1)
security_delay.head()

### d. NAS delay

In [ ]:
nas_delay =  df[df['NAS_DELAY'] > 0]
nas_delay = nas_delay.reset_index()

nas_delay = nas_delay.drop('index', axis=1)
nas_delay.head()

### e. Late aircraft delay

In [ ]:
aircraft_delay =  df[df['LATE_AIRCRAFT_DELAY'] > 0]
aircraft_delay = aircraft_delay.reset_index()

aircraft_delay = aircraft_delay.drop('index', axis=1)
aircraft_delay.head()

### Bar chart for causes of delay

In [ ]:
cause_names  = ['aircraft_delay', 'security_delay', 'carrier_delay', 'nas_delay','weather_delay']
count_values = [len(aircraft_delay), len(security_delay), len(carrier_delay), len(nas_delay), len(weather_delay)]

causes_of_delay = pd.DataFrame({'CAUSES': cause_names, 'COUNTS': count_values})

x = range(len(cause_names))

plt.figure(figsize=(6,8))
sns.barplot(x='CAUSES', y='COUNTS', data = causes_of_delay)

for i in x:
    plt.annotate(str(count_values[i]), xy=(x[i], count_values[i]), ha='center', va='bottom', size=14)

plt.ylabel('NUMBER OF DELAYED ARRIVAL FLIGHTS (ARRIVAL DELAY > 0 minutes)')
plt.xlabel('CAUSE OF DELAY')
plt.xticks(rotation=70, size=12)
plt.tight_layout()
plt.show()

Histograms of delayed arrival flights with respect to the causes of delay (January and February 2017). Most of delayed arrival flights are caused by NAS, aircraft and carrier (airline). Security caused a small number of delayed flights.

## 5.5 Arrival delay with respect to day of week

In [ ]:
arrival_delay_dow = df[ df['ARR_DELAY'] > 0 ]
arrival_delay_dow = arrival_delay_dow.groupby(['DAY_OF_WEEK'])['ARR_DELAY'].count().reset_index(name='COUNTS')
arrival_delay_dow

In [ ]:
arrival_delay_dow['DAY_OF_WEEK'] = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday' ]

plt.figure(figsize=(8,6))
sns.barplot(x='DAY_OF_WEEK', y='COUNTS', data=arrival_delay_dow)
plt.ylabel('NUMBER OF DELAYED ARRIVAL FLIGHTS')
plt.xlabel('DAY OF WEEK', size = 14)
plt.xticks(rotation=70, size = 12)
plt.tight_layout()
plt.show()

Histograms of delayed arrival flights with respect to the day of week (January and February 2017). Most of flights arrived late on Mondays, Thursdays, Fridays and Sundays. Saturdays see fewer delayed flights.

## 5.6 Arrival delay with respect to day of month

In [ ]:
arrival_delay_dom = df[ df['ARR_DELAY'] > 0 ]
arrival_delay_dom = arrival_delay_dom.groupby(['DAY_OF_MONTH'])['ARR_DELAY'].count().reset_index(name='COUNTS')
arrival_delay_dom

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(x='DAY_OF_MONTH', y='COUNTS', data=arrival_delay_dom)
plt.ylabel('NUMBER OF DELAYED ARRIVAL FLIGHTS')
plt.xlabel('DAY OF MONTH')
plt.xticks(rotation=70)
plt.tight_layout()
plt.show()

Histograms of delayed arrival flights with respect to day of month (January and February 2017). There were fewer delayed flights in the middle and in the end of the months.

## 5.7 Average delay of 20 busiest airports

In [ ]:
# Top 20 busiest destinations
top20_dest = list(df['DEST'].value_counts().head(20).index)
print(top20_dest)

top20_dest_df = df[df['DEST'].isin(top20_dest)][['OP_UNIQUE_CARRIER','ORIGIN','DEST','ARR_DELAY', 'AIRLINE_NAME']]
top20_dest_df.head()

In [ ]:
delays = (pd.DataFrame(top20_dest_df.groupby('DEST', as_index=False)['ARR_DELAY']
                       .mean())
          .sort_values(by='ARR_DELAY', ascending=False))
delays

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot(x='ARR_DELAY', y='DEST', data=delays, palette='flare')
ax.set_title('Average Arrival Delay in Minutes of 20 Busiest Aiports', size=14)
ax.set_ylabel('')
ax.set_xlabel('Average Arrival Delay [Minutes]', size=14)
fig = ax.get_figure()

for bar in ax.patches:
    ax.text(bar.get_width()-0.75,  bar.get_y()+bar.get_height()/2. + 0.15,
           '{:1.2f}'.format(bar.get_width()), ha='center', color='b', size='11', weight='bold')

SFO airport has a longest arrival delay with a mean of 24 minutes, the flights arrived at MCO are late only about 30 seconds. The flights landed at CLT aiport were about 3 minute early.

## 5.8 Arrival Delay for the top 20 airlines

I will next visualize the arrival delays for the top 20 airlines. The top 20 airlines are defined as the airlines with the top 20 most flights in the dataset.

In [ ]:
# Top 12 busiest airlines
top20_airlines = list(df['OP_UNIQUE_CARRIER'].value_counts().head(20).index)
print(top20_airlines)

top20_airlines_df = df[df['OP_UNIQUE_CARRIER'].isin(top20_airlines)][['OP_UNIQUE_CARRIER','ORIGIN','DEST','ARR_DELAY', 'AIRLINE_NAME']]
top20_airlines_df.head()

Out of the 20 airlines that have had the most flights, let's take a look at who has had delays greater than 30 minutes.

In [ ]:
temp_df = top20_airlines_df.copy()

# Get delays greater than 30 minutes
temp_df['ARR_DELAY'] = temp_df['ARR_DELAY'].apply(lambda x: 30 if x >=30 else x)
temp_df['ARR_DELAY'] = temp_df['ARR_DELAY'].apply(lambda x: -30 if x <= -30 else x)

In [ ]:
plt.figure(figsize=(20,5))
ax = sns.violinplot(x='AIRLINE_NAME', y='ARR_DELAY', data=temp_df)
ax.set_ylabel('Arrival Delay [minutes]')
ax.set_xlabel('Airlines')
label = ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
fig = ax.get_figure()

The median for the top 20 airlines is around 0 to -10 minutes, meaning that most flights arrive before their scheduled time. The width of the plots at 30 represents the frequency of flights arriving 30 minutes or more after the scheduled arrival time.

Next, we will plot the percentage of all flights delayed more than 30 minutes.

In [ ]:
# flights delayed more than 30 minutes.
delays = pd.DataFrame()

for uc in list(df['OP_UNIQUE_CARRIER'].unique()):
    percent_delay = len(df[(df['OP_UNIQUE_CARRIER']==uc) & (df['ARR_DELAY'] >= delay_thresh)].index)/len(df[df['OP_UNIQUE_CARRIER']==uc].index)*100
    delays = delays.append({'OP_UNIQUE_CARRIER': uc, 'PERCENT_DELAY': percent_delay}, ignore_index=True)

delays = pd.merge(delays, carriers, how='left', left_on='OP_UNIQUE_CARRIER', right_on='code')
delays.drop(['OP_UNIQUE_CARRIER', 'code'], axis=1, inplace=True)
delays.rename(index=str, columns={'Description':'AIRLINE'}, inplace=True)
delays.sort_values(by=['PERCENT_DELAY'], ascending=False, inplace=True)
delays

In [ ]:
# Plot
plt.figure(figsize=(10,8))
ax = sns.barplot(x='PERCENT_DELAY', y='AIRLINE_NAME', data=delays, palette='flare')

ax.set_title('Percent of arrival flights delayed more than 30 minutes', size=14)
ax.set_ylabel('')
ax.set_xlabel('[%]', size=14)
fig = ax.get_figure()

for bar in ax.patches:
    ax.text(bar.get_width()-0.9,  bar.get_y()+bar.get_height()/2. + 0.15,
           '{:1.2f}%'.format(bar.get_width()), ha='center', color='white', size='11', weight='bold')

Percent of arrival flights delayed more than 30 minutes. Among the top 12 busiest airlines, Virgin America has the largest ratio of delayed flights with 22\%, the Hawaiian Ailiines Inc. has the lowest ratio.

## 5.9 Delay per Airline on the Destination Airports

In [ ]:
# Params for plots
font = {'family' : 'normal', 'weight' : 'bold', 'size'   : 15}
sns.set_style('whitegrid')

# Initialize the Graph
fig_dim=(16,10)
fig, ax =plt.subplots(figsize=fig_dim)
sns.despine(bottom=True, left=True)

# Draw each observation with a scatterplot
sns.stripplot(x='ARR_DELAY', y='AIRLINE_NAME', hue='AIRLINE_NAME',data=df , size=4,  linewidth = 0.5,  jitter=True)
plt.xlabel('Flight Arrival Delay on the Destination Airports')
plt.ylabel('AirLines')
ax.set_xticklabels(['{:2.0f}h{:2.0f}m'.format(*[int(y) for y in divmod(x,60)])
                         for x in ax.get_xticks()])
ax.yaxis.label.set_visible(True)

Sactter plot of delay per Airline on the Destination Airports. Among the top 12 busiest airlines, Virgin America still has the longest of delay up to 33h20m late, the Alaska Ailiines Inc. has the shortest delays up to 8h20m.

## 5.10 Origin/Destination Analysis

Next, I'll perform some analysis on the origin and destination airports. First, find the top 10 busiest airports in terms of flight frequency. The ranking of airports are found to be consistent when counting for both origin and destination, and for each airport, the number of incoming and outgoing flights are roughly equal (as seen below).

In [ ]:
origin_airport_counts = df['ORIGIN'].value_counts().to_dict()

origin_airport_counts = sorted(origin_airport_counts.items(), key=lambda x: x[1], reverse=True)
print( origin_airport_counts[:10] )

top10_origin_airports = [x[0] for x in origin_airport_counts[:10]]
top10_origin_airports_counts = [x[1] for x in origin_airport_counts[:10]]
print(top10_origin_airports)

del origin_airport_counts

In [ ]:
dest_airport_counts = df['DEST'].value_counts().to_dict()

dest_airport_counts = sorted(dest_airport_counts.items(), key=lambda x: x[1], reverse=True)
print( dest_airport_counts[:10] )

top10_dest_airports = [x[0] for x in dest_airport_counts[:10]]
top10_dest_airports_counts = [x[1] for x in dest_airport_counts[:10]]
print(top10_dest_airports)

del dest_airport_counts

In [ ]:
print( top10_origin_airports ==  top10_dest_airports)

In [ ]:
df_top10_airports = pd.DataFrame( {'Origin': top10_origin_airports_counts, 
                                  'Destination': top10_dest_airports_counts},
                                 index=top10_origin_airports)

df_top10_airports.plot.bar(rot=0, xlabel='Airports', ylabel='Count', figsize=(10,7), title='Counts of inbound/outbound flights for 10 busiest airports')

del df_top10_airports

Atlanta Airport (ATL) is the busiest airport, with over 50,000 incoming and outgoing flights over the 2-month period.

For each airport, the number of incoming and outgoing flights are roughly equal 

In [ ]:
airport_pairs = df.groupby(['ORIGIN', 'DEST']).agg({'DEP_DELAY': ['mean', 'min', 'max', 'count']})
airport_pairs.columns = ['DEP_DELAY_MEAN', 'DEP_DELAY_MIN', 'DEP_DELAY_MAX', 'COUNTS']
airport_pairs = airport_pairs.reset_index()

airport_pairs = airport_pairs[airport_pairs.COUNTS > 20]

print('Parameters of ORIGIN-DESTINATION airport pairs')
print('Number of airport pairs considered: ', len(airport_pairs))

airport_pairs.sort_values(by='DEP_DELAY_MEAN', ascending=False)

ORIGIN-DESTINATION airport pairs with highest and lowest average delay. Only take into account the airport pairs that had at least 20 flights over this 2-month period (average of 10 flights per month) as I wanted to consider flights that were relatively frequent.

Highest Average Delay: 77.2 minutes (late) from LGA to CAE

Lowest Average Delay: -33 minutes (early) from RHI to IMT

## 5.11 Analysis (t-test & Mann-Whitney U test for Hypothesis testing)

**Test if the means of arrival delays (in minutes) in January and February 2017 are equal?**

The arrival delay in minutes for 2 months (January and February).

In [ ]:
delayed_jan = df.ARR_DELAY[ ( (df.ARR_DELAY > 0) & (df.MONTH == 1) )] 
print( 'Number of samples in January: ', len(delayed_jan) )
delayed_jan.describe()

In [ ]:
delayed_feb = df.ARR_DELAY[ ( (df.ARR_DELAY > 0) & (df.MONTH == 2) )] 
print( 'Number of samples in February: ',len(delayed_feb) )
delayed_feb.describe()

In [ ]:
plt.figure(figsize=(10,8))
sns.distplot(delayed_jan,  kde=False, label='Jan')
sns.distplot(delayed_feb,  kde=False, label='Feb')
plt.title("Sample T-Test")
plt.xlim(-100, 300)
plt.xlabel('Arrival delay in minutes')
plt.ylabel('Count')
plt.legend()
plt.show()

Histograms of delayed arrival flights (arrival delay > 0 minutes) for January anf Febuary of 2017

#### 5.11a Independent Sample T-Test

Define the Null and Alternate Hypothesis:


*   Null Hypothesis: the means of both groups are equal (µ1 = µ2)
*   Alternate Hypothesis: the means of both group are not equal (µ1 ≠ µ2)



In [ ]:
from scipy import stats
t_stat, p_value = stats.ttest_ind(delayed_jan, delayed_feb, equal_var = False)
print('t-statistic:{0}, p-value:{1}'.format(t_stat, p_value)) # p-Value and the T-Statistic
# P-Value:8.536872289699007e-138 T-Statistic:25.00028507345877

There is enough evidence to reject the Null Hypothesis of equal means as the P-Value is low (typically ≤ 0.05).

Statistically speaking, there is enough evidence to conclude that the population means are not equal.

#### 5.11b Mann-Whitney U Test

Define the Null and Alternate Hypothesis:


*   Null Hypothesis: the means of both groups are equal (µ1 = µ2)
*   Alternate Hypothesis: the means of both group are not equal (µ1 ≠ µ2)



In [ ]:
#perform the Mann-Whitney U test
u_stat, p_value = stats.mannwhitneyu(delayed_jan, delayed_feb, alternative='two-sided')
print('u-statistic:{0}, p-value:{1}'.format(u_stat, p_value)) # p-Value and the U-Statistic

There is enough evidence to reject the Null Hypothesis of equal means as the P-Value is low (typically ≤ 0.05).